In [1]:
import os
import obspy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, ConvLSTM2D
%matplotlib inline
%run helperfunctions.ipynb
os.getcwd()

'/Users/albertkong/grillochallenge'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir('lse_challenge/data')

In [4]:
# Signal List
os.chdir('signal')
signals = []
signal_files = os.listdir()
for sigfile in signal_files[0:200]: #Need to remove index for actual dataset
    tmp_trace = obspy.read(sigfile)
    cutlength = np.random.randint(900)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 1])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 1])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 1])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 1])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 1])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 1])
    

# Noise List
os.chdir('../noise')
noise_files = os.listdir()
for noisefile in noise_files[0:200]: #Need to remove index for actual dataset
    tmp_trace = obspy.read(noisefile)
    cutlength = np.random.randint(900)
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 0])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 0])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 0])
    y_mat = rot_y(trace_mat, theta)
    signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 0])
    z_mat = rot_z(trace_mat, theta)
    signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 0])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 0])
    
os.chdir('../')

sig_df = pd.DataFrame(signals, columns = ["tr_x", "tr_y", "tr_z", "magnitude", "cutlength", "signal"])
sig_df['p_arrival'] = 1000 - sig_df['cutlength']

In [5]:
#sig_df.head()

In [6]:
features = []
targets = []

for index, slice_df in sig_df.iterrows():
    x = slice_df["tr_x"]
    y = slice_df["tr_y"]
    z = slice_df["tr_z"]
    mag = slice_df["magnitude"]
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    d = {"x": x, "y":y, "z":z, "magnitude":mag}
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(25).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(50).mean()
    temp_df["STV"] = temp_df["magnitude"].rolling(25).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(50).var()
    temp_df["pwave"] = 0
    temp_df["pwave"].loc[p_arrival:(p_arrival + 25)] = 1
    
    temp_df.dropna(inplace = True)
    
    features.append(temp_df.drop(columns = ["pwave"]).values)
    targets.append(signal)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.3)

In [8]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
feature_shape = X_train[0].shape

In [19]:
model = Sequential()
model.add(LSTM(32, input_shape = feature_shape))
model.add(LSTM(32, input_shape = feature_shape))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

ValueError: Input 0 of layer "lstm_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 32)

In [10]:
model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.fit(X_train, y_train, epochs = 30)

Epoch 1/30


2022-03-02 17:41:29.436140: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-02 17:41:30.194344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 17:41:30.359990: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 17:41:33.395560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 39s 460ms/step - loss: 0.4949 - accuracy: 0.7018
Epoch 2/30
53/53 [==============================] - 11s 207ms/step - loss: 0.6082 - accuracy: 0.7911
Epoch 3/30
53/53 [==============================] - 11s 205ms/step - loss: 0.5506 - accuracy: 0.6988
Epoch 4/30
53/53 [==============================] - 11s 208ms/step - loss: 0.6508 - accuracy: 0.6393
Epoch 5/30
53/53 [==============================] - 11s 203ms/step - loss: 0.5850 - accuracy: 0.7851
Epoch 6/30
53/53 [==============================] - 11s 207ms/step - loss: 0.5997 - accuracy: 0.6768
Epoch 7/30
53/53 [==============================] - 11s 204ms/step - loss: 0.5509 - accuracy: 0.7351
Epoch 8/30
53/53 [==============================] - 11s 202ms/step - loss: 0.5190 - accuracy: 0.7417
Epoch 9/30
53/53 [==============================] - 11s 208ms/step - loss: 0.4652 - accuracy: 0.8202
Epoch 10/30
53/53 [==============================] - 11s 205ms/step - loss: 0.4225 - accuracy: 0.8363


In [11]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose = 0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-02 17:47:33.334879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 17:47:33.467899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.9183517098426819, Accuracy: 0.5888888835906982


In [12]:
predictions = model.predict(X_test)
predictions = predictions > 0.5
f1_score = sklearn.metrics.f1_score(y_test, predictions)
print(f"F1 Score: {f1_score}")

2022-03-02 17:47:42.682272: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-02 17:47:42.721195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


F1 Score: 0.7034068136272545


In [13]:
predictions = model.predict(X_train)
predictions = predictions > 0.5
f1_score = sklearn.metrics.f1_score(y_train, predictions)
print(f"F1 Score: {f1_score}")

F1 Score: 0.7150696496411988
